In [ ]:
from Tkinter import *
from tkFileDialog import askopenfilename, asksaveasfilename
import re
import tarfile
import ipaddress

def show_entry_fields():
    def pathOpen():
        Tk().withdraw()
        return askopenfilename()  # show an "Open" dialog box and return the path to the selected file

    def pathSave():
        ftypes = [('Normal text file', 'txt')]
        path = asksaveasfilename(filetypes=ftypes, defaultextension='.txt')
        return path

    def unTar():
        tar = tarfile.open(pathOpen())
        f = tar.extractfile('./support.txt')  # Extracts the support.txt
        content = f.read().replace('\r\n', '\n')  # reads the content of the support.txt
        content = content.replace('\\\n', '')
        tar.close()
        return content

    config = unTar().split("!Device Configuration")[1]
    config = config.split("security certificate table Name:")[0]  # cutting the Configuration out of the support file.

    ip = "%s" % str(ipaddress.ip_address(unicode(e1.get())))
    mask = "%s" % (e2.get())
    interface = "%s" % (e3.get())
    gw = "%s" % str(ipaddress.ip_address(unicode(e4.get())))

    defaultSnmp = """manage snmp groups create SNMPv1 public -gn initial
manage snmp groups create SNMPv1 ReadOnlySecurity -gn InitialReadOnly
manage snmp groups create SNMPv2c public -gn initial
manage snmp groups create SNMPv2c ReadOnlySecurity -gn InitialReadOnly
manage snmp groups create UserBased radware -gn initial
manage snmp groups create UserBased ReadOnlySecurity -gn InitialReadOnly
manage snmp access create initial SNMPv1 noAuthNoPriv -rvn iso -wvn iso \
-nvn iso
manage snmp access create InitialReadOnly SNMPv1 noAuthNoPriv -rvn \
ReadOnlyView
manage snmp access create initial SNMPv2c noAuthNoPriv -rvn iso -wvn iso \
-nvn iso
manage snmp access create InitialReadOnly SNMPv2c noAuthNoPriv -rvn \
ReadOnlyView
manage snmp access create initial UserBased authPriv -rvn iso -wvn iso \
-nvn iso
manage snmp access create InitialReadOnly UserBased authPriv -rvn \
ReadOnlyView
manage snmp views create iso 1
manage snmp views create ReadOnlyView 1
manage snmp views create ReadOnlyView 1.3.6.1.4.1.89.2.7.2 -cm excluded
manage snmp views create ReadOnlyView 1.3.6.1.6.3.18.1.1 -cm excluded
manage snmp views create ReadOnlyView 1.3.6.1.6.3.15.1.2.2 -cm excluded
manage snmp views create ReadOnlyView 1.3.6.1.4.1.89.35.1.61 -cm \
excluded
manage snmp views create ReadOnlyView 1.3.6.1.6.3.16.1.2 -cm excluded
manage snmp views create ReadOnlyView 1.3.6.1.6.3.16.1.4 -cm excluded
manage snmp views create ReadOnlyView 1.3.6.1.6.3.16.1.5 -cm excluded
manage snmp notify create allTraps -ta v3Traps
manage snmp users create radware -cf 0.0 -ap MD5 -akc \
9579279544ecd9324d5a0ad17e55ec4d -pp DES -pkc \
9579279544ecd9324d5a0ad17e55ec4d
manage snmp target-address create v3MngStations -tl v3Traps -p \
radware-authPriv
manage snmp target-parameters create public-v1 -d SNMPv1 -sm SNMPv1 -sn \
public -sl noAuthNoPriv
manage snmp target-parameters create public-v2 -d SNMPv2c -sm SNMPv2c -sn \
public -sl noAuthNoPriv
manage snmp target-parameters create radware-authPriv -d SNMPv3 -sm \
UserBased -sn radware -sl authPriv
manage snmp community create public -n public -sn public"""

    config = re.sub(r'net ip-interface create [0-9].*',
                    "net ip-interface create " + ip + ' ' + mask + ' ' + interface, config)
    config = re.sub(r'net route table create [0-9].*',
                    "net route table create 0.0.0.0 0.0.0.0 " + gw + ' -i ' + interface, config)
    config = re.sub(r'manage snmp [a-z].*', '', config)
    config = re.sub(r'manage user table .*', '', config)

    config = config + "%s" % str(defaultSnmp)


    open(pathSave(), 'wb').write(config)

# master = Tk()
# master.title("UpConfig")
# master.minsize(300, 120)
# 
# Label(master, text="SNMP Version: ").grid(row=0)
# Label(master, text="User/Community Name: ").grid(row=1)
# Label(master, text="Use Authentication: ").grid(row=2)
# Label(master, text="Authentication Password: ").grid(row=3)
# Label(master, text="Use Privacy: ").grid(row=4)
# Label(master, text="Privacy Password: ").grid(row=5)
# 
# 
# e1 = Entry(master)
# e2 = Entry(master)
# e3 = Entry(master)
# e4 = Entry(master)
# 
# 
# 
# # e1.insert(1, "SNMPv1")
# # e2.insert(1, "radware")
# # e3.insert(1, "radware")
# # e4.insert(1, "10.176.254.254")
# 
# e1.grid(row=0, column=1)
# e2.grid(row=1, column=1)
# e3.grid(row=2, column=1)
# e4.grid(row=3, column=1)
# 
# Button(master, text='Quit', command=master.quit, fg="red", bg="yellow").grid(row=4, column=0, sticky=N, pady=4)
# Button(master, text='Go !', command=show_entry_fields, fg="red", bg="yellow").grid(row=4, column=1, sticky=N, pady=4)


mainloop()
